In [61]:
import pandas as pd
import numpy as np

In [31]:
DATAPATH_2015 = '../data/targetstation46240/46240h2015.txt'
DATAPATH_2016 = '../data/targetstation46240/46240h2015.txt'
DATAPATH_2019 = '../data/targetstation46240/46240h2019.txt'

# Format 2019 - 2022 Historical Data

For the baseline model, we will only use wave related data as features. We have also downloaded ocean current data from the same buoy location, however, we do not want to over over-featurize given the total size of the dataset and MLP model.

Most of the functions in this section format the data text files into dataframes, handle the type conversions, and drop columns we will not use. First, we begin with creating a function `text_to_dataframe` that reads in a datapath and converts it to a pd dataframe.

In [63]:
def text_to_dataframe(data_path):
    """
    Converts a text file at the given path into a pandas DataFrame.

    Args:
        data_path (str): The path to the data file.

    Returns:
        tuple: A tuple where the first element is a pandas Series containing the units 
               of each column, and the second element is the pandas DataFrame with 
               the data.
    """
    # Read the data file into a DataFrame
    historical_data = pd.read_csv(data_path, delim_whitespace=True, header=0)

    # Extract the units row and drop it from the DataFrame
    units = historical_data.iloc[0]
    historical_data.drop(0, inplace=True)

    # Reset the index of the DataFrame
    historical_data.reset_index(drop=True, inplace=True)

    return units, historical_data

There is a little data cleaning we need to do here. As you can see below (where we look at the heads of the dataframes), the values for certain columns are all 99.0 or 9999.0. These values signal a missing value. In the case of our data, the buoy likely does not have the proper instruments to capture these values. To fix this, we will simply drop the columns.

Another issue is that we just loaded in a text file, so all the values in the dataframe are strings. The models we train later in the script need numerical data. We can use some useful functions from the pandas library to convert it to a digestable format for our future model.

To make window slicing easier for the time series data, I only keep measurements on the start of each hour. The original dataframes contain measurements for every 30 minutes. Given that there are marginal changes (from initial observations) every 30 minutes, we only consider inputs that are seperated by one hour of time for the baseline models. We accomplish this with the last function in the block below.

We also drop certain columns to simplify our feature set for the baseline mode. For detailed descriptions of each feature, please refer [here](https://www.ndbc.noaa.gov/faq/measdes.shtml#stdmet).

In [29]:
def drop_cols(dataframe, columns_to_drop):
    """
    Drop the given columns_to_drop in the dataframe
    """
    dataframe.drop(columns=columns_to_drop, inplace=True)

def convert_to_numerical_data(dataframe):
    """
    Convert all columns to numerical values, since our data is passed
    in as a .txt
    """
    # Convert all columns to numerical values
    dataframe.apply(pd.to_numeric)

    # Convert specific columns to integers
    columns_to_convert_to_int = ['#YY', 'MM', 'DD', 'hh', 'mm']
    for col in columns_to_convert_to_int:
        dataframe[col] = dataframe[col].astype(int)

def convert_to_hourly(dataframe):
    """
    Convert our data to an hourly format.
    """
    # Drop rows where 'mm' column is not 0
    drop_indices = dataframe[dataframe['mm'] != 0].index
    dataframe.drop(drop_indices, inplace=True)

For each year, we seem to have some missing values (see the smaller number of rows for the dataframe compared to 2020 and 2022). The missing values likely reflect certain times when the sensors of the buoy were malfuncitoning. Because we are dealing with time series data, where datapoints need to be ordered sequentially, we need to account for these missing rows when processing the dataset.

The function below is responsible for finding where the measurement between the ith row and i-1th row is greater than 60 minutes. We will have to account for these rows when preprocessing the data.

Note: We check for 60 minute differences after dropping all measurements that are not taken on the hour! See the other functions below to get a better idea of how we accomplish this.

In [56]:
def check_for_missing_rows(dataframe):
    """
    Identifies missing rows in a DataFrame based on time intervals.

    Args:
        dataframe (pd.DataFrame): The DataFrame to check. It must contain columns 
                                  named '#YY', 'MM', 'DD', 'hh', and 'mm', representing 
                                  the year, month, day, hour, and minute, respectively.

    Returns:
        list: A list of indices where the DataFrame has missing rows based on the time 
              difference between consecutive rows.
    """
    # Rename columns for compatibility with pd.to_datetime()
    dataframe = dataframe.rename(columns={
        '#YY': 'year',
        'MM': 'month',
        'DD': 'day',
        'hh': 'hour',
        'mm': 'minute'
    })

    # Convert the separate year, month, day, hour, minute columns into a single datetime column
    dataframe['datetime'] = pd.to_datetime(dataframe[['year', 'month', 'day', 'hour', 'minute']])

    missing_indices = []

    # Iterate over rows and check for a time difference of more than 30 minutes
    for i in range(1, dataframe.shape[0]):
        time_diff = dataframe.iloc[i]['datetime'] - dataframe.iloc[i-1]['datetime']
        if time_diff > pd.Timedelta(minutes=60):
            missing_indices.append(i)

    return missing_indices

Now, we need to split the data into time series training and label targets. 

For this step, we need to decide our forecasting window. Given that we are attempting to build an open source wave forecasting model for surfers, we will aim to predict the mean significant wave height in 24-48 hours.

Let's start with a 24 hour lead time. With this decided, we can format a dataset for supervised learning. Our x's will be vectors containg the set of features we specify measured at each hour for a certain amount hours. In the code below, we have 4 features measured at each hour for 24 hours. Thus, each x_i will be 96 x 1 ((4 * 24) x 1). The corresponding label will just be a scalar representing the 'WVHT' value 24 hours ahead of the last hour measurements were taken in the x_i vector.
We choose our feature set to be wave height (WVHT), dominant wave period (DPD), 

In [59]:
def build_supervised_learning_dataset(dataframe, missing_indices, feature_set=['WVHT', 'DPD', 'APD', 'MWD'], window=24, lead=24):
    # Convert missing_indices to a set for faster lookup
    missing_set = set(missing_indices)

    X, y = [], []

    for i in range(dataframe.shape[0] - window - lead + 1):
        # Check if any of the indices in the current slice are in the missing set
        if any([(i + j) in missing_set for j in range(window)]):
            continue
        
        # Here we grab the window of values for each feature
        x_i = dataframe.iloc[i : i + window][feature_set].to_numpy()

        # Reshape the array so it can be a row vector of X (1 X 24 * 4) -> (1 x 96)
        x_i = x_i.flatten()

        # Only fetch the significant wave height value for the label
        y_i = dataframe.iloc[i + window + lead - 1]['WVHT']

        X.append(x_i)
        y.append(y_i)

    return np.array(X), np.array(y)

### 2019

We will now explore our data and further clean it before we attempt to predict. We will begin with 2019 and do it cell by cell, but at the end we will generalize and launch it on all our different years.

In [70]:
_, historical_data_2019 = text_to_dataframe(DATAPATH_2019)

print(f'shape: {historical_data_2019.shape}')

shape: (17299, 18)


We drop WDIR, WSPD, GST, PRES, ATMP, DEWP, VIS, and TIDE because this dataset does not contain those values. TODO: add other dataset to combine with this one in order to fill in those values.

In [69]:
drop_cols(historical_data_2019, ['WDIR', 'WSPD', 'GST', 'PRES', 'ATMP', 'DEWP', 'VIS', 'TIDE'])

convert_to_numerical_data(historical_data_2019)

# Later, could convert to every 4hr or 12hr
convert_to_hourly(historical_data_2019)

# See first 24 hours of dataset
historical_data_2019.head(24)

,#YY,MM,DD,hh,mm,WVHT,DPD,APD,MWD,WTMP
0,2019,1,1,0,0,1.46,11.11,6.56,346,13.6
2,2019,1,1,1,0,1.44,11.76,6.40,351,13.5
4,2019,1,1,2,0,1.33,9.88,6.38,343,13.5
6,2019,1,1,3,0,1.24,9.88,6.63,341,13.5
8,2019,1,1,4,0,1.08,10.53,6.20,345,13.5
10,2019,1,1,5,0,1.11,9.88,6.98,339,13.4
12,2019,1,1,6,0,1.15,9.88,7.40,342,13.4
14,2019,1,1,7,0,1.06,9.09,7.19,334,13.4
16,2019,1,1,8,0,1.09,11.11,7.45,343,13.4
18,2019,1,1,9,0,0.94,9.88,7.18,338,13.4


In [57]:
missing_indices_2019 = check_for_missing_rows(historical_data_2019)

len(missing_indices_2019)

81

In [62]:
X_2019, y_2019 = build_supervised_learning_dataset(historical_data_2019, missing_indices_2019)

### 2015 and other years

In [74]:
def preprocess_data(path_to_data, buoy_specs):
    _, historical_data = text_to_dataframe(path_to_data)

    drop_cols(historical_data, buoy_specs)

    convert_to_numerical_data(historical_data)
    
    # Later, could convert to every 4hr or 12hr
    convert_to_hourly(historical_data)
    
    # See first 24 hours of dataset
    print(historical_data.head(24))

    missing_indices = check_for_missing_rows(historical_data)
    len(missing_indices)

    X, y = build_supervised_learning_dataset(historical_data, missing_indices)
    return X_2019, y_2019

In [75]:
X_2015, y_2015 = preprocess_data(DATAPATH_2015, ['WDIR', 'WSPD', 'GST', 'PRES', 'ATMP', 'DEWP', 'VIS', 'TIDE'])

Empty DataFrame
Columns: [#YY, MM, DD, hh, mm, WVHT, DPD, APD, MWD, WTMP]
Index: []
